# Extracción de los datos

Luego de descargar de la página del [INDEC](https://www.indec.gob.ar/indec/web/Nivel4-Tema-3-5-32) los archivos con los valores del Índice de Precios Internos al por Mayor (IPIM), correspondientes a los períodos 1996-2015 y 2016-2024, procedemos a armar un .csv con los valores correspondientes al período 2012-2019.

Estos datos fueron descargados el 17/10/2024, utilizando el script `utils/descarga-ipim.py`.

## 2012 - 2015

Como el archivo con los datos correspondientes al período 2012-2015 están disponibles en un archivo de Excel, nos pareció más sencillo directamente extrar los valores del ipim_general "a mano" utilizando LibreOffice. 
El archivo resultante se encuentra en `data/ipim/ipim_2012_2015_base_1993.csv`.

### Interpolación de datos faltantes

Como no se encontraron los valores de este índice para los meses de noviembre y diciembre de 2015, procedimos a estimarlos a través de la interpolación.

In [118]:
import pandas as pd
from scipy.interpolate import interp1d
import numpy as np

file_path = "data/ipim/ipim_2012_2015_base_1993.csv"
ipim_base_1993_oct_2015 = pd.read_csv(file_path, on_bad_lines='warn')

ipim_base_1993_oct_2015['periodo'] = pd.to_datetime(
    ipim_base_1993_oct_2015['periodo'])

# Agregar los meses de noviembre y diciembre de 2015
nov_dec_2015 = pd.DataFrame({
    'periodo': [pd.to_datetime('2015-11-01'), pd.to_datetime('2015-12-01')]
})
ipim_base_1993_dec_2015 = pd.concat(
    [ipim_base_1993_oct_2015, nov_dec_2015], ignore_index=True)

# Extraer los datos válidos para la interpolación
valid_data = ipim_base_1993_dec_2015.dropna(subset=['indice_ipim'])

# Crear la función de interpolación
interp_func = interp1d(
    valid_data['periodo'].astype(np.int64),
    valid_data['indice_ipim'],
    kind='linear',
    fill_value='extrapolate'
)

# Interpolar los datos faltantes
ipim_base_1993_dec_2015['indice_ipim'] = ipim_base_1993_dec_2015['indice_ipim'].combine_first(
    pd.Series(
        interp_func(ipim_base_1993_dec_2015['periodo'].astype(np.int64)),
        index=ipim_base_1993_dec_2015.index
    )
)

print(ipim_base_1993_dec_2015.tail())

      periodo  indice_ipim
43 2015-08-01      909.850
44 2015-09-01      922.140
45 2015-10-01      930.690
46 2015-11-01      939.525
47 2015-12-01      948.075


### Cambio de base

Para simplificar la interpretación y la comparacion de los datos procedemos a tomar el valor de enero de 2012 como referencia.

$$ \text{Dato de IPIM base 2012} = \frac{\text{Dato de IPIM base 1993}}{\text{IPIM de ene 2012 base 1993}} * 100 $$

In [131]:
ipim_enero_2012 = ipim_base_1993_dec_2015[ipim_base_1993_dec_2015['periodo']
                                          == pd.to_datetime('2012-01-01')].iloc[0]['indice_ipim']


def cambio_de_base_enero_2012(row):
    return (row['indice_ipim'] / ipim_enero_2012) * 100


ipim_base_2012_dec_2015 = ipim_base_1993_dec_2015.copy()
ipim_base_2012_dec_2015['indice_ipim'] = ipim_base_2012_dec_2015.apply(
    cambio_de_base_enero_2012, axis=1)

print("Head:")
print(ipim_base_2012_dec_2015.head())
print("\nTail:")
print(ipim_base_2012_dec_2015.tail())

Head:
     periodo  indice_ipim
0 2012-01-01   100.000000
1 2012-02-01   100.987732
2 2012-03-01   102.181241
3 2012-04-01   103.337514
4 2012-05-01   104.391879

Tail:
      periodo  indice_ipim
43 2015-08-01   178.311057
44 2015-09-01   180.719633
45 2015-10-01   182.395249
46 2015-11-01   184.126720
47 2015-12-01   185.802336


## 2016 - 2019

### Preparación de los datos

In [123]:
import pandas as pd

file_path = 'data/ipim/indice_ipim_2016_2024.csv'
ipim_base_2015_sep_2024 = pd.read_csv(file_path, delimiter=';', decimal=',')

# Ajustar el formato de las columnas
ipim_base_2015_sep_2024 = ipim_base_2015_sep_2024.rename(
    columns={'indice': 'indice_ipim'})

ipim_base_2015_sep_2024['periodo'] = pd.to_datetime(
    ipim_base_2015_sep_2024['periodo'], format='%Y-%m-%d')

# Filtrar los datos de nivel general
ipim_base_2015_sep_2024 = ipim_base_2015_sep_2024[
    ipim_base_2015_sep_2024['nivel_general_aperturas'] == 'ng_nivel_general']

# Filtrar los datos hasta diciembre de 2019
ipim_base_2015_dec_2019 = ipim_base_2015_sep_2024[ipim_base_2015_sep_2024['periodo']
                                                  <= pd.to_datetime('2019-12-01')]

# Eliminar la columna de nivel general
ipim_base_2015_dec_2019 = ipim_base_2015_dec_2019.drop(
    columns=['nivel_general_aperturas'])

print(ipim_base_2015_dec_2019.tail())

      periodo  indice_ipim
44 2019-08-01   372.785790
45 2019-09-01   388.320934
46 2019-10-01   402.345280
47 2019-11-01   424.027901
48 2019-12-01   439.656651


### Cambio de base

Para pasar los datos de 2016 a 2019 con base de diciembre de 2015 a enero 2012, utilizamos la regla de tres simple:

$$ \text{Factor de empalme} = \frac{\text{Dato de IPIM diciembre 2015 base 2012}}{\text{Dato de IPIM diciembre 2015 base 2015}}$$ 

$$ \text{Dato de 2016 a 2019 base 2012 } = \text{Dato de IPIM base 2015} * \text{factor de empalme} $$

In [132]:
# Calcular el factor de empalme
ipim_diciembre_2015_base_2012 = ipim_base_2012_dec_2015[ipim_base_2012_dec_2015['periodo']
                                                        == pd.to_datetime('2015-12-01')].iloc[0]['indice_ipim']

ipim_diciembre_2015_base_2015 = ipim_base_2015_dec_2019[ipim_base_2015_dec_2019['periodo']
                                                        == pd.to_datetime('2015-12-01')].iloc[0]['indice_ipim']

factor_de_empalme = ipim_diciembre_2015_base_2012 / ipim_diciembre_2015_base_2015


def cambio_de_base_2015_a_2012(row):
    return row['indice_ipim'] * factor_de_empalme


# Aplicar el cambio de base
ipim_base_2012_dec_2015_dec_2019 = ipim_base_2015_dec_2019.copy()
ipim_base_2012_dec_2015_dec_2019['indice_ipim'] = ipim_base_2015_dec_2019.apply(
    cambio_de_base_2015_a_2012, axis=1)

print("Head:")
print(ipim_base_2012_dec_2015_dec_2019.head())
print("\nTail:")
print(ipim_base_2012_dec_2015_dec_2019.tail())

Head:
     periodo  indice_ipim
0 2015-12-01   185.802336
1 2016-01-01   202.129120
2 2016-02-01   212.132170
3 2016-03-01   217.389937
4 2016-04-01   220.756542

Tail:
      periodo  indice_ipim
44 2019-08-01   692.644707
45 2019-09-01   721.509366
46 2019-10-01   747.566929
47 2019-11-01   787.853745
48 2019-12-01   816.892328


## 2012 - 2019

### Concatenación y guardado de los dos períodos

In [133]:
ipim_base_2012_ene_2012_dec_2019 = pd.concat(
    [ipim_base_2012_dec_2015, ipim_base_2012_dec_2015_dec_2019], ignore_index=False)

# Eliminar duplicado de diciembre de 2015
ipim_base_2012_ene_2012_dec_2019.drop_duplicates(
    subset='periodo', inplace=True)

print("Head:")
print(ipim_base_2012_ene_2012_dec_2019.head())
print("\nTail:")
print(ipim_base_2012_ene_2012_dec_2019.tail())

# Guardar los datos
ipim_base_2012_ene_2012_dec_2019.to_csv(
    'data/ipim/ipim_mensual_2012-2019_base_ene_2012.csv', index=False)

Head:
     periodo  indice_ipim
0 2012-01-01   100.000000
1 2012-02-01   100.987732
2 2012-03-01   102.181241
3 2012-04-01   103.337514
4 2012-05-01   104.391879

Tail:
      periodo  indice_ipim
44 2019-08-01   692.644707
45 2019-09-01   721.509366
46 2019-10-01   747.566929
47 2019-11-01   787.853745
48 2019-12-01   816.892328


### IPIM anualizado

Cómo decidímos trabajar la base de datos de presupuesto agrupada anualmente, calculamos el IPIM promediado anualmente para aplicarlo en a los valores de presupuesto.

In [134]:
ipim_mensual = pd.read_csv(
    'data/ipim/ipim_mensual_2012-2019_base_ene_2012.csv')
ipim_mensual['periodo'] = pd.to_datetime(ipim_mensual['periodo'])

ipim_anual = ipim_mensual.groupby(
    pd.Grouper(key='periodo', freq='YE')).mean().reset_index()

ipim_anual['periodo'] = ipim_anual['periodo'].dt.to_period('Y')

ipim_anual.to_csv('data/ipim/ipim_anual_2012-2019_base_ene_2012.csv', index=False)

print(ipim_anual)

  periodo  indice_ipim
0    2012   105.998249
1    2013   120.444969
2    2014   153.000921
3    2015   174.864122
4    2016   232.412607
5    2017   273.601249
6    2018   409.455699
7    2019   651.589741
